In [ ]:
pip install netCDF4

In [1]:
import netCDF4
import xarray as xr
import numpy as np
import pandas as pd


In [3]:
file_path_nc ="D:/Masters/Thesis/Data/Combined_FLDAS.nc"
dataset = xr.open_dataset(file_path_nc)
dataset


C:\Users\Hp\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


<xarray.Dataset>
Dimensions:                (time: 251, bnds: 2, X: 160, Y: 110)
Coordinates:
  * time                   (time) datetime64[ns] 2003-02-01 ... 2023-12-01
  * X                      (X) float64 76.05 76.15 76.25 ... 91.75 91.85 91.95
  * Y                      (Y) float64 21.05 21.15 21.25 ... 31.75 31.85 31.95
Dimensions without coordinates: bnds
Data variables:
    time_bnds              (time, bnds) datetime64[ns] ...
    Evap_tavg              (time, Y, X) float32 ...
    Qs_tavg                (time, Y, X) float32 ...
    SWE_inst               (time, Y, X) float32 ...
    Tair_f_tavg            (time, Y, X) float32 ...
    SoilMoi00_10cm_tavg    (time, Y, X) float32 ...
    SoilMoi10_40cm_tavg    (time, Y, X) float32 ...
    SoilMoi40_100cm_tavg   (time, Y, X) float32 ...
    SoilMoi100_200cm_tavg  (time, Y, X) float32 ...
Attributes: (12/20)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    Conventions:               CF-1.4
    history:                   Fri Oct 30 12:23:53 2020: cdo -z zip_4 replace...
    institution:               NASA GSFC
    missing_value:             -9999.0
    NUM_SOIL_LAYERS:           4
    ...                        ...
    DX:                        0.1
    DY:                        0.1
    nco_openmp_thread_number:  1
    NCO:                       4.4.4
    history_L34RS:             'Created by L34RS v1.4.4 @ NASA GES DISC on Ju...
    CDO:                       Climate Data Operators version 1.9.8 (https://...

# Regrid using DataFRame

In [ ]:
which = 'NDVI'
var = dataset.metpy.parse_cf(which)
test = var.isel(time=0)
df1 = pd.DataFrame(data=test.values)
df1

## Specially for LST

In [ ]:
# Extract the variable of interest
which = 'SWE_inst'
var = dataset.metpy.parse_cf(which)

# Extract longitude and latitude values
lons = dataset.metpy.parse_cf('X')
lats = dataset.metpy.parse_cf('Y')
var
if lats[0] > lats[-1]:
    lats = lats[::-1]
    var = var[:, ::-1, :]
dataset['y'] = lats
unit = var.units
dataset

## Regridding using dataarray

In [5]:
regrid_lat = np.arange(21.125,32.125,0.25)
regrid_lon = np.arange(76.125,92.125,0.25)
data_array = dataset.interp(Y = regrid_lat, X = regrid_lon,method='slinear')
#data_array - data_array.rename({'X':'lon','Y':'lat'})
data_array

<xarray.Dataset>
Dimensions:                (time: 251, bnds: 2, Y: 44, X: 64)
Coordinates:
  * time                   (time) datetime64[ns] 2003-02-01 ... 2023-12-01
  * Y                      (Y) float64 21.12 21.38 21.62 ... 31.38 31.62 31.88
  * X                      (X) float64 76.12 76.38 76.62 ... 91.38 91.62 91.88
Dimensions without coordinates: bnds
Data variables:
    time_bnds              (time, bnds) datetime64[ns] 2003-02-01 ... 2024-01-01
    Evap_tavg              (time, Y, X) float64 3.01e-06 6.039e-06 ... 8.543e-07
    Qs_tavg                (time, Y, X) float64 7.997e-10 ... 7.326e-10
    SWE_inst               (time, Y, X) float64 0.0 0.0 0.0 ... 0.2052 0.1048
    Tair_f_tavg            (time, Y, X) float64 298.8 297.8 ... 262.9 261.0
    SoilMoi00_10cm_tavg    (time, Y, X) float64 0.1501 0.1748 ... 0.1767 0.1777
    SoilMoi10_40cm_tavg    (time, Y, X) float64 0.1491 0.1566 ... 0.2639 0.2771
    SoilMoi40_100cm_tavg   (time, Y, X) float64 0.1501 0.157 ... 0.2009 0.1917
    SoilMoi100_200cm_tavg  (time, Y, X) float64 0.3851 0.3833 ... 0.16 0.2002
Attributes: (12/20)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    Conventions:               CF-1.4
    history:                   Fri Oct 30 12:23:53 2020: cdo -z zip_4 replace...
    institution:               NASA GSFC
    missing_value:             -9999.0
    NUM_SOIL_LAYERS:           4
    ...                        ...
    DX:                        0.1
    DY:                        0.1
    nco_openmp_thread_number:  1
    NCO:                       4.4.4
    history_L34RS:             'Created by L34RS v1.4.4 @ NASA GES DISC on Ju...
    CDO:                       Climate Data Operators version 1.9.8 (https://...

## Rename x and y to lat and lon if reuired

In [7]:
data_array = data_array.rename({'X':'lon','Y':'lat'})
data_array

<xarray.Dataset>
Dimensions:                (time: 251, bnds: 2, lat: 44, lon: 64)
Coordinates:
  * time                   (time) datetime64[ns] 2003-02-01 ... 2023-12-01
  * lat                    (lat) float64 21.12 21.38 21.62 ... 31.38 31.62 31.88
  * lon                    (lon) float64 76.12 76.38 76.62 ... 91.38 91.62 91.88
Dimensions without coordinates: bnds
Data variables:
    time_bnds              (time, bnds) datetime64[ns] 2003-02-01 ... 2024-01-01
    Evap_tavg              (time, lat, lon) float64 3.01e-06 ... 8.543e-07
    Qs_tavg                (time, lat, lon) float64 7.997e-10 ... 7.326e-10
    SWE_inst               (time, lat, lon) float64 0.0 0.0 ... 0.2052 0.1048
    Tair_f_tavg            (time, lat, lon) float64 298.8 297.8 ... 262.9 261.0
    SoilMoi00_10cm_tavg    (time, lat, lon) float64 0.1501 0.1748 ... 0.1777
    SoilMoi10_40cm_tavg    (time, lat, lon) float64 0.1491 0.1566 ... 0.2771
    SoilMoi40_100cm_tavg   (time, lat, lon) float64 0.1501 0.157 ... 0.1917
    SoilMoi100_200cm_tavg  (time, lat, lon) float64 0.3851 0.3833 ... 0.2002
Attributes: (12/20)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    Conventions:               CF-1.4
    history:                   Fri Oct 30 12:23:53 2020: cdo -z zip_4 replace...
    institution:               NASA GSFC
    missing_value:             -9999.0
    NUM_SOIL_LAYERS:           4
    ...                        ...
    DX:                        0.1
    DY:                        0.1
    nco_openmp_thread_number:  1
    NCO:                       4.4.4
    history_L34RS:             'Created by L34RS v1.4.4 @ NASA GES DISC on Ju...
    CDO:                       Climate Data Operators version 1.9.8 (https://...

# Regridding using xCDAT

In [ ]:
 conda create --clone base --prefix /home/jupyter/newenvironment -y 

In [ ]:
!conda install xcdat
import xcdat as xc
ds = xr.open_dataset(file_path_nc)
ds

## Assess the regridded Product

In [ ]:
#for variable in data_array.variables:
 #   print(variable)
which = 'Qs_tavg'
var = dataset.metpy.parse_cf(which)
values = var.isel(time=0)
values= values.fillna(-9999)
lons = dataset.metpy.parse_cf('lon')
lats = dataset.metpy.parse_cf('lat')
values

## See Regridded Data in DAtaFRAme

In [ ]:
df = pd.DataFrame(data=values.values)
df

## Export Regridded data to CSV File

In [ ]:
df = values.to_dataframe().reset_index()
df

In [ ]:
# Convert the data to a DataFrame
df = values.to_dataframe().reset_index()

# Pivot the DataFrame
pivot_df = df.pivot(index='lat', columns='lon', values=which)

# Display the pivoted DataFrame
pivot_df

# Specify the output file path
output_csv_path = "D:/Masters/Thesis/Data/ReGrid/"+"Regrid"+which+".csv"

# Export the pivoted DataFrame to a CSV file
pivot_df.to_csv(output_csv_path)

print(f"Pivoted DataFrame has been exported to {output_csv_path}")

## Save the Regridded data into .nc file for FLDAS specifically by averaging for several depths

In [ ]:
# Save the regridded data to a NetCDF file
import xarray as xr
vars = [variable for variable in data_array.variables]
print(vars)
dynamic_vars = {}
for i in range(6, 10):
    dynamic_vars["iget" + vars[i]] = data_array[vars[i]]
average_data_array = None

# Loop through each DataArray in the dictionary and compute the mean
for key, da in dynamic_vars.items():
    if average_data_array is None:
        average_data_array = da
    else:
        average_data_array += da
average_data_array /= len(dynamic_vars)
average_data_array.name = "SM"
average_data_array
output_file = "D:/Masters/Thesis/Data/ReGrid/SM.nc"
#ds.to_netcdf(output_file)
average_data_array.to_netcdf(output_file)

## Save the Regridded data into .nc file

In [11]:
# Save the regridded data to a NetCDF file
import xarray as xr
which = 'Evap_tavg'
output_file = "D:/Masters/Thesis/Data/ReGrid/"+which+".nc"
#exporting a specific variable from data_array
subset = data_array[which]
subset.to_netcdf(output_file)
#ds.to_netcdf(output_file)
#data_array.to_netcdf(output_file)
subset

<xarray.DataArray 'Evap_tavg' (time: 251, lat: 44, lon: 64)>
array([[[3.00998482e-06, 6.03909945e-06, 5.26596409e-06, ...,
                    nan,            nan,            nan],
        [6.19164942e-06, 4.29995214e-06, 4.70788973e-06, ...,
                    nan,            nan,            nan],
        [7.12235990e-06, 5.93499243e-06, 4.91008299e-06, ...,
                    nan,            nan,            nan],
        ...,
        [1.01021493e-05, 1.26630040e-05, 1.26986571e-05, ...,
         1.00726959e-06, 8.22274366e-07, 7.76586869e-07],
        [9.90336679e-06, 9.95632968e-06, 9.94623497e-06, ...,
         7.03028672e-07, 6.44171823e-07, 7.54848809e-07],
        [           nan, 1.04298323e-05, 8.71085751e-06, ...,
         1.20561691e-06, 1.29832130e-06, 4.37959768e-06]],

       [[1.24997992e-06, 2.14204855e-06, 1.77119013e-06, ...,
                    nan,            nan,            nan],
        [2.22770066e-06, 1.71262021e-06, 1.64185026e-06, ...,
                    nan,            nan,            nan],
        [3.13075519e-06, 2.50171135e-06, 1.77617309e-06, ...,
                    nan,            nan,            nan],
...
        [1.88484782e-05, 2.44414573e-05, 2.73653945e-05, ...,
         1.27304008e-06, 2.25504637e-06, 2.19628060e-06],
        [2.54275112e-05, 2.67887266e-05, 2.68579141e-05, ...,
         9.50851959e-07, 2.61281309e-06, 3.27801322e-06],
        [           nan, 2.76167189e-05, 2.59579350e-05, ...,
         1.18231117e-06, 2.23800058e-06, 2.64250413e-06]],

       [[1.98757119e-05, 1.77480109e-05, 1.89972070e-05, ...,
                    nan,            nan,            nan],
        [1.87134132e-05, 1.97152498e-05, 1.92786888e-05, ...,
                    nan,            nan,            nan],
        [1.81238750e-05, 1.90224588e-05, 1.99025567e-05, ...,
                    nan,            nan,            nan],
        ...,
        [1.37205319e-05, 1.69796764e-05, 1.87160847e-05, ...,
         9.84887802e-07, 1.09985632e-06, 1.61119082e-06],
        [1.69814917e-05, 1.75222696e-05, 1.78284624e-05, ...,
         5.12332450e-07, 1.00515353e-06, 1.23719933e-06],
        [           nan, 1.76940463e-05, 1.71643605e-05, ...,
         6.90850079e-07, 1.11035734e-06, 8.54275481e-07]]])
Coordinates:
  * time     (time) datetime64[ns] 2003-02-01 2003-03-01 ... 2023-12-01
  * lat      (lat) float64 21.12 21.38 21.62 21.88 ... 31.12 31.38 31.62 31.88
  * lon      (lon) float64 76.12 76.38 76.62 76.88 ... 91.12 91.38 91.62 91.88
Attributes:
    long_name:      total evapotranspiration
    standard_name:  total_evapotranspiration
    vmax:           1000000000000000.0
    cell_methods:   time:mean
    units:          kg m-2 s-1
    vmin:           -1000000000000000.0

## Plot the Data

In [ ]:

file_path_nc =output_file
data_array = xr.open_dataset(file_path_nc)
var = data_array.metpy.parse_cf(which)
values = var.isel(time=0)
lons = data_array.metpy.parse_cf('lon')
lats = data_array.metpy.parse_cf('lat')
unit = var.units
# Plot the data
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
# Add coastlines and borders for context
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plot the data
c = ax.pcolormesh(lons, lats, values, cmap='viridis', transform=ccrs.PlateCarree())
# Add a colorbar
fig.colorbar(c, ax=ax, orientation='vertical', label=unit)

# Add title and labels
ax.set_title(which)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

## Plot several graphs (FOR X AND Y)

In [ ]:
import matplotlib.pyplot as plt
lst = xr.open_dataset("D:/Masters/Thesis/Data/ReGrid/Qs_tavg.nc")
display(lst)
time = lst['time']
#display(time)
# Assuming lst is an xarray Dataset
lst = lst.transpose('time', 'lat', 'lon')  # Transpose if necessary
# Check if lat/lon are correctly labeled, otherwise rename them
lst = lst.rename({'lat': 'lat', 'lon': 'lon'})
for yr in range(2005,2006):
  filt_time = time.sel(time=time.dt.year==yr)
  display(filt_time)
  fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10))
  axes = axes.flatten()
  i=1
  # Plot each month
  for t in filt_time:
      monthly_data = lst.sel(time=t)
      #print(t)
      # Assuming you want to plot the first variable in the dataset
      monthly_data['Qs_tavg'].plot(ax=axes[i-1])
      #axes[i].set_title(f'Month {i}')
      i+=1

  # Adjust layout
  plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, hspace=0.4, wspace=0.4)
  #plt.savefig(f'/content/drive/MyDrive/Thesis_folder/pre_processed_data/IMERG/{yr}.png', bbox_inches='tight', dpi=300)
  plt.show()

## Plot for lon and lat

In [ ]:
import matplotlib.pyplot as plt
lst = xr.open_dataset("D:/Masters/Thesis/Data/NDVI/NDVI_2003_2023.nc")
display(lst)
time = lst['time']
#display(time)
# Assuming lst is an xarray Dataset
lst = lst.transpose('time', 'lat', 'lon')  # Transpose if necessary
# Check if lat/lon are correctly labeled, otherwise rename them
lst = lst.rename({'lat': 'lat', 'lon': 'lon'})
for yr in range(2005,2006):
  filt_time = time.sel(time=time.dt.year==yr)
  display(filt_time)
  fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10))
  axes = axes.flatten()
  i=1
  # Plot each month
  for t in filt_time:
      monthly_data = lst.sel(time=t)
      #print(t)
      # Assuming you want to plot the first variable in the dataset
      monthly_data['NDVI'].plot(ax=axes[i-1],cmap = 'viridis')
      #axes[i].set_title(f'Month {i}')
      i+=1

  # Adjust layout
  plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, hspace=0.4, wspace=0.4)
  #plt.savefig(f'/content/drive/MyDrive/Thesis_folder/pre_processed_data/IMERG/{yr}.png', bbox_inches='tight', dpi=300)
  plt.show()


## Count number of NAN

In [ ]:
nan_count = lst.isnull().sum()
print(f'Total NaN values in the dataset: \n{nan_count}')